**Import TensorFlow and other libraries**

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

**Download the Shakespeare dataset**

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


**Read the data**

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

65 unique characters


**Process the text**

In [6]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [7]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [8]:
# Show how the first 13 characters from the text are mapped to integers
print('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


**Create training examples and targets**

In [9]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [13]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


**Create training batches**

In [14]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

**Build The Model**

In [15]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [17]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

**Try the model**

In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [20]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [21]:
sampled_indices

array([50,  2, 31,  6, 34, 30, 26, 13, 14,  8, 61, 17,  3, 28, 43, 10, 39,
       25, 35, 53, 13,  6, 56, 29, 43,  9, 49, 33, 59, 39, 42, 37, 25, 53,
       35, 48, 59, 33, 27, 48, 43, 31,  2, 58,  9,  7, 61, 63, 14, 55, 13,
       21, 39, 63, 53, 59, 64, 42, 33, 35, 55, 50, 45, 56, 10, 60,  6,  9,
       21,  4, 58, 53, 51, 35,  8, 51, 36, 21, 36, 35, 35, 28,  5,  5, 48,
       23, 31, 31, 42, 59, 62,  4, 53, 36, 54, 38, 41, 39, 14, 63])

In [22]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 ' tyrant;\nAnd rather proved the sliding of your brother\nA merriment than a vice.\n\nISABELLA:\nO, pardon'

Next Char Predictions: 
 "l!S,VRNAB.wE$Pe:aMWoA,rQe3kUuadYMoWjuUOjeS!t3-wyBqAIayouzdUWqlgr:v,3I&tomW.mXIXWWP''jKSSdux&oXpZcaBy"


**Train the model**

In [23]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1733656


In [24]:
model.compile(optimizer='adam', loss=loss)

**Configure checkpoints**

In [25]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

**Execute the training**

In [26]:
EPOCHS = 10

In [27]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 880s 5s/step - loss: 2.6917
Epoch 2/10
172/172 [==============================] - 874s 5s/step - loss: 1.9596
Epoch 3/10
172/172 [==============================] - 886s 5s/step - loss: 1.6907
Epoch 4/10
172/172 [==============================] - 888s 5s/step - loss: 1.5420
Epoch 5/10
172/172 [==============================] - 882s 5s/step - loss: 1.4537
Epoch 6/10
172/172 [==============================] - 882s 5s/step - loss: 1.3934
Epoch 7/10
172/172 [==============================] - 886s 5s/step - loss: 1.3478
Epoch 8/10
172/172 [==============================] - 880s 5s/step - loss: 1.3085
Epoch 9/10
172/172 [==============================] - 874s 5s/step - loss: 1.2741
Epoch 10/10
172/172 [==============================] - 877s 5s/step - loss: 1.2410


**Generate the text**

In [28]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [29]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


**The Predication Summary**

In [31]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [32]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: I cannot make gone;
As mine are nursed, with in his body's pale,
And now, outrage wrecked; 'tis away.
I'll make a strange of some faults, sir, as you say do not fear.

TRANIO:
Sir I:
That news as sweetaling sufferance to equit,
Whome-maristops and riptles, those are enter'd:
But what say thy father is my cell,
Thou did charget knowledge spee like Henry.

Lord:
Never show begina witness, sir, dogns our gail.

KING RICHARD III:
Here conjusts pot thy hand: Comfort, moe befall, I pray?
Go love, I'll one things, buy as a noble form.
What pleasant he is a gentleman:
Nay, but rempath you so so.

POLIXENS:
Not at your highness' lord guiler two
Which thou shalt smocts our strange; disbreased it is:
he suns me go with ame: to let him
When they shall be free bless-tizens?

CATESBY:

Citizens:
Who knows not the cetter I reseech you,
Joind to the citcheth of this lowful head!

QUEEN MARGARET:
No measure, sir, wholks, at awly, you're put them
To your life cares. Now, I beseech you, love, is h

**Advanced: Customized Training**

In [33]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [34]:
optimizer = tf.keras.optimizers.Adam()

In [35]:
@tf.function
def train_step(inp, target):
    with tf.GradientTape() as tape:
        predictions = model(inp)
        loss = tf.reduce_mean(
            tf.keras.losses.sparse_categorical_crossentropy(
                target, predictions, from_logits=True))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss

In [36]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()

    # resetting the hidden state at the start of every epoch
    model.reset_states()

    for (batch_n, (inp, target)) in enumerate(dataset):
        loss = train_step(inp, target)

        if batch_n % 100 == 0:
            template = 'Epoch {} Batch {} Loss {}'
            print(template.format(epoch + 1, batch_n, loss))

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print('Epoch {} Loss {:.4f}'.format(epoch + 1, loss))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.174066543579102
Epoch 1 Batch 100 Loss 2.3446543216705322
Epoch 1 Loss 2.1371
Time taken for 1 epoch 897.8744790554047 sec

Epoch 2 Batch 0 Loss 2.1336426734924316
Epoch 2 Batch 100 Loss 1.9290683269500732
Epoch 2 Loss 1.7896
Time taken for 1 epoch 877.5726189613342 sec

Epoch 3 Batch 0 Loss 1.751135230064392
Epoch 3 Batch 100 Loss 1.673738718032837
Epoch 3 Loss 1.5923
Time taken for 1 epoch 892.8444237709045 sec

Epoch 4 Batch 0 Loss 1.6366620063781738
Epoch 4 Batch 100 Loss 1.514611840248108
Epoch 4 Loss 1.4775
Time taken for 1 epoch 883.0398263931274 sec

Epoch 5 Batch 0 Loss 1.4380489587783813
Epoch 5 Batch 100 Loss 1.4721332788467407
Epoch 5 Loss 1.4421
Time taken for 1 epoch 878.3726303577423 sec

Epoch 6 Batch 0 Loss 1.3592214584350586
Epoch 6 Batch 100 Loss 1.3682396411895752
Epoch 6 Loss 1.4280
Time taken for 1 epoch 873.6201405525208 sec

Epoch 7 Batch 0 Loss 1.3047558069229126
Epoch 7 Batch 100 Loss 1.370040774345398
Epoch 7 Loss 1.3784
Time taken for 